[Look Up](https://www.luogu.org/problemnew/show/P2947)。给定一数组，求各数字右边第一个比该数字大的数，没有则设置0.

思路：单调栈的典型应用。从后往前扫描数组，设立一个栈，栈中始终保存比当前数字大的元素，若有小的全部弹出。

In [1]:
def LookUp(nums):
    n = len(nums)
    res = [0]*n
    s = list()

    for idx in range(n-1, -1, -1):
        # 先排空栈中不大于当前数的所有数字
        while s and nums[s[-1]] <= nums[idx]:
            s.pop()

        # 栈中剩下的元素都是比当前大的数了
        if not s:    # 栈为空
            res[idx] = 0
        else:    # 栈不空，比当前大的最近的数就是栈顶元素
            res[idx] = s[-1]

        s.append(idx)

    return res

[Remove K Digits](https://leetcode.com/problems/remove-k-digits/)。给一个数字字串，删掉其中的$k$位，使得得到的数字最小。

思路：单调栈。所有位顺序入栈，易得高位数字越小越好，当某一位小于栈顶元素时，需要弹栈。注意弹栈的次数不能超过$k$次。不难，实现逻辑稍微复杂一点。

In [2]:
def removeKdigits(num: str, k: int) -> str:
    n = len(num)
    if k >= n:
        return '0'

    s = list()
    cnt = 0

    for ch in num:
        if cnt < k:    # 删除k次
            if not s:
                s.append(ch)
                continue

            while s and int(ch) < int(s[-1]) and cnt < k:
                s.pop()
                cnt += 1
            s.append(ch)

        else:
            s.append(ch)

    return str(int(''.join(s[:n-k])))

猿辅导2019笔试题。给$C$个字符串的压缩表示形式，对其进行解码复原。

思路：输入字串可能带括号也可能没带括号，都需要考虑进去。

In [4]:
def func(C):
    def reverse(s):
        s = list(s)
        i, j = 0, len(s) - 1
        while i < j:
            s[i], s[j] = s[j], s[i]
            i += 1
            j -= 1
        return ''.join(s)

    for _ in range(C):
        s = sys.stdin.readline().strip()
        n = len(s)
        stack = list()
        num = ''
        idx = 0

        while idx < n:
            # 字母与括号入栈
            while idx < n and not s[idx].isdigit():
                stack.append(s[idx])
                idx += 1

            # 提取数字
            while idx < n and s[idx].isdigit():
                num += s[idx]
                idx += 1
            num = int(num) if num != '' else 1

            sub_s = ''    # 被压缩的子串
            if stack[-1] == ')':    # 有括号时需特殊处理
                left_cnt = 0
                while stack and stack[-1] != '(':
                    while stack[-1] == ')':
                        left_cnt += 1
                        stack.pop()
                    while stack and stack[-1] != '(':
                        sub_s += stack.pop()
                for _ in range(left_cnt):    # 清空等数量的左括号
                    stack.pop()
            else:    # 无括号时取出单个字符即可
                sub_s = stack.pop()

            sub_s = reverse(sub_s) * num
            num = ''
            for ch in sub_s:
                stack.append(ch)

        res = ''
        while stack[-1] == ')':
            stack.pop()
        while stack and stack[-1] != '(':
            if stack[-1] == ')':
                stack.pop()
                continue
            res += stack.pop()
        print(reverse(res))

[Decode String](https://leetcode.com/problems/decode-string/)。给一个编码压缩后的字串，字符被包裹在方括号中，数字表示方括号中字符的重复次数，根据该规则还原完整的字符串。

思路：纯逻辑题，写起来比较麻烦。当遇到右括号时，说明需要弹栈，直到弹到左括号为止，这一部分是压缩的字串；再弹出栈中所有的数字。

In [2]:
def decodeString(s: str) -> str:
    stack = list()

    for ch in s:
        if ch != ']':
            stack.append(ch)
        else:
            # 1. 弹出压缩的字串
            enc_s = str()
            while stack[-1] != '[':
                enc_s = stack.pop()+enc_s
            _ = stack.pop()

            # 2. 弹出数字
            num = str()
            while stack and stack[-1].isdigit():
                num = stack.pop()+num
            num = int(num)

            # 3. 还原
            dec_s = enc_s*num
            for c in dec_s:
                stack.append(c)

    return ''.join(stack)


decodeString("3[a]2[bc]")

'aaabcbc'

[Min Stack](https://leetcode.com/problems/min-stack/)。设计一个栈结构，要求其```push()```、```top()```、```pop()```、```min()```的时间复杂度均为$O(1)$。

思路：设置一个辅助栈，里面只压入$min$值。弹栈时，当普通栈与辅助栈的栈顶元素相同时，辅助栈才需要弹栈。

In [ ]:
class MinStack:
    def __init__(self):
        """
        initialize your data structure here.
        """
        self.s=list()
        self.s_min=list()

    def push(self, x: int) -> None:
        self.s.append(x)
        if not self.s_min or x<=self.s_min[-1]:
            self.s_min.append(x)

    def pop(self) -> None:
        if self.s_min[-1]==self.s[-1]:
            self.s_min.pop()
        self.s.pop()

    def top(self) -> int:
        return self.s[-1]

    def getMin(self) -> int:
        return self.s_min[-1]

[Dota2 Senate](https://leetcode.com/problems/dota2-senate/)。给一只含'R'和'D'的字串，表示两阵营的攻击顺序。每一阵营成员攻击会杀死一名另一阵营的成员，求胜利阵营。

思路：设置两个队列，双方成员均进入队列，为每一位成员标上一个索引，索引越小攻击顺序越靠前。每次从两队列中取出成员PK，只有胜者才会再次进入队列并更新索引。

In [3]:
def predictPartyVictory(senate: str) -> str:
    q_D, q_R = list(), list()
    n = len(senate)

    for idx, ch in enumerate(senate):
        if ch == 'R':
            q_R.append(idx)
        else:
            q_D.append(idx)

    while q_D and q_R:
        idx_D, idx_R = q_D.pop(0), q_R.pop(0)
        if idx_D < idx_R:
            q_D.append(idx_D+n)
        else:
            q_R.append(idx_R+n)

    return "Radiant" if q_R else 'Dire'

'Dire'